## Install the latest .whl package

Check [here](https://pypi.org/project/semantic-link-labs/) to see the latest version.

In [ ]:
%pip install semantic-link-labs

## Import the library and set initial parameters

In [ ]:
import sempy_labs as labs
from sempy_labs import migration, directlake
import sempy_labs.report as rep
import time

datamart_workspace_name        = ''             # Specify the name of the workspace containing the datamart semantic model.
datamart_dataset_name          = ''             # Specify the name of the datamart semantic model.
semantic_model_workspace_name  = ''             # Specify the name of the workspace containing the new semantic model will be stored. This can be the same as the datamart workspace.
semantic_model_name            = ''             # Specify the name of the new Direct Lake semantic model.
item_workspace_name            = ''             # Specify the name of the workspace containing the data warehouse that the datamart was migrated to.
item_name                      = ''             # Specify the name of the warehouse that the datamart was migrated to.
schema_name                    = ''             # Specify the name of the schema where the data is stored in the data warehouse.
item_type                      = 'Warehouse'    # Specify the item type for the migration; options are 'Warehouse' or 'Lakehouse'.

# Function: v2_migrate_tables_columns_to_semantic_model

Includes support of multiple item types in Fabric including the options: Lakehouse and Warehouse

In [ ]:
import pandas as pd
import sempy.fabric as fabric
from sempy_labs._helper_functions import resolve_lakehouse_name, retry
from sempy_labs.directlake import generate_shared_expression
from sempy_labs.lakehouse._lakehouse import lakehouse_attached
from sempy_labs.tom import connect_semantic_model
from typing import Optional
from sempy._utils._log import log
import sempy_labs._icons as icons

@log
def v2_migrate_tables_columns_to_semantic_model(
    dataset: str,
    new_dataset: str,
    workspace: Optional[str] = None,
    new_dataset_workspace: Optional[str] = None,
    item_name: Optional[str] = None,
    item_workspace: Optional[str] = None,
    item_type: Optional[str] = 'Lakehouse',
    schema_name: Optional[str] = None
):
    """
    Adds tables/columns to the new Direct Lake semantic model based on an import/DirectQuery semantic model.

    Parameters
    ----------
    dataset : str
        Name of the import/DirectQuery semantic model.

    new_dataset : str
        Name of the Direct Lake semantic model.

    workspace : str, default=None
        The Fabric workspace name in which the import/DirectQuery semantic model exists.
        Defaults to None which resolves to the workspace of the attached lakehouse
        or if no lakehouse attached, resolves to the workspace of the notebook.

    new_dataset_workspace : str
        The Fabric workspace name in which the Direct Lake semantic model will be created.
        Defaults to None which resolves to the workspace of the attached lakehouse
        or if no lakehouse attached, resolves to the workspace of the notebook.

    item_name : str, default=None
        The item name used by the Direct Lake semantic model.
        Defaults to None which resolves to the lakehouse attached to the notebook.

    item_workspace : str, default=None
        The Fabric workspace used by the item.
        Defaults to None which resolves to the workspace of the attached lakehouse
        or if no lakehouse attached, resolves to the workspace of the notebook.

    item_type : str, default=lakehouse
        The item type that the Direct Lake semantic model will be created from.
        Defaults to lakehouse, supported items include [Lakehouse, Warehouse].

    """

    if dataset == new_dataset:
        raise ValueError(
            f"{icons.red_dot} The 'dataset' and 'new_dataset' parameters are both set to '{dataset}'. These parameters must be set to different values."
        )

    # Array for supported item types
    item_types = ["Lakehouse", "Warehouse"]
    item_type = item_type.capitalize()
    if item_type not in item_types:
        raise ValueError(
            f"{icons.red_dot} Invalid item type. Valid options: {item_types}."
        )

    workspace = fabric.resolve_workspace_name(workspace)

    if new_dataset_workspace is None:
        new_dataset_workspace = workspace

    if item_workspace is None:
        item_workspace = new_dataset_workspace

    shEx = generate_shared_expression(item_name, item_type, item_workspace)

    dfC = fabric.list_columns(dataset=dataset, workspace=workspace)
    dfT = fabric.list_tables(dataset=dataset, workspace=workspace)
    dfT.rename(columns={"Type": "Table Type"}, inplace=True)
    dfC = pd.merge(
        dfC,
        dfT[["Name", "Table Type"]],
        left_on="Table Name",
        right_on="Name",
        how="left",
    )
    dfT_filt = dfT[dfT["Table Type"] == "Table"]
    dfC_filt = dfC[
        (dfC["Table Type"] == "Table")
        & ~(dfC["Column Name"].str.startswith("RowNumber-"))
        & (dfC["Type"] != "Calculated")
    ]

    print(f"{icons.in_progress} Updating '{new_dataset}' based on '{dataset}'...")

    @retry(
        sleep_time=1,
        timeout_error_message=f"{icons.red_dot} Function timed out after 1 minute",
    )
    def dyn_connect():
        with connect_semantic_model(
            dataset=new_dataset, readonly=True, workspace=new_dataset_workspace
        ) as tom:

            tom.model

    dyn_connect()

    with connect_semantic_model(
        dataset=new_dataset, readonly=False, workspace=new_dataset_workspace
    ) as tom:
        if not any(e.Name == "DatabaseQuery" for e in tom.model.Expressions):
            tom.add_expression("DatabaseQuery", expression=shEx)
            print(f"{icons.green_dot} The 'DatabaseQuery' expression has been added.")

        for i, r in dfT_filt.iterrows():
            tName = r["Name"]
            tDC = r["Data Category"]
            tHid = bool(r["Hidden"])
            tDesc = r["Description"]
            ent_name = tName
            for char in icons.special_characters:
                ent_name = ent_name.replace(char, "")
            if schema_name != None:
                tSLT = f"[{schema_name}].[{tName}]"
            if not any(t.Name == tName for t in tom.model.Tables):
                tom.add_table(
                    name=tName,
                    description=tDesc,
                    data_category=tDC,
                    hidden=tHid,
                    source_lineage_tag=tSLT
                )
                tom.add_entity_partition(table_name=tName, entity_name=ent_name, schema_name=schema_name)
                print(f"{icons.green_dot} The '{tName}' table has been added.")

        for i, r in dfC_filt.iterrows():
            tName = r["Table Name"]
            cName = r["Column Name"]
            scName = r["Source"]
            cHid = bool(r["Hidden"])
            cDataType = r["Data Type"]
            for char in icons.special_characters:
                scName = scName.replace(char, "")

            if scName.endswith("_"):
                scName = scName[:-1]

            if not any(
                c.Name == cName and c.Parent.Name == tName for c in tom.all_columns()
            ):
                tom.add_data_column(
                    table_name=tName,
                    column_name=cName,
                    source_column=scName,
                    hidden=cHid,
                    data_type=cDataType,
                )
                print(
                    f"{icons.green_dot} The '{tName}'[{cName}] column has been added."
                )

        print(
            f"\n{icons.green_dot} All regular tables and columns have been added to the '{new_dataset}' semantic model."
        )

## Create the Direct Lake model based on the datamart semantic model


In [ ]:
labs.create_blank_semantic_model(dataset = semantic_model_name, workspace = semantic_model_workspace_name)

v2_migrate_tables_columns_to_semantic_model(
    workspace               = datamart_workspace_name,
    dataset                 = datamart_dataset_name,
    new_dataset_workspace   = semantic_model_workspace_name,
    new_dataset             = semantic_model_name,
    item_workspace          = item_workspace_name,
    item_name               = item_name,
    item_type               = item_type,
    schema_name             = schema_name
)

migration.migrate_model_objects_to_semantic_model(
    workspace               = datamart_workspace_name,
    dataset                 = datamart_dataset_name,
    new_dataset_workspace   = semantic_model_workspace_name,
    new_dataset             = semantic_model_name
)

migration.migrate_field_parameters(
    workspace               = datamart_workspace_name,
    dataset                 = datamart_dataset_name,
    new_dataset_workspace   = semantic_model_workspace_name,
    new_dataset             = semantic_model_name
)

time.sleep(10)

labs.refresh_semantic_model(workspace = semantic_model_workspace_name, dataset = semantic_model_name)

migration.refresh_calc_tables(workspace = semantic_model_workspace_name, dataset = semantic_model_name)

labs.refresh_semantic_model(workspace = semantic_model_workspace_name, dataset = semantic_model_name)

## Show migrated/unmigrated objects

In [ ]:
migration.migration_validation(
    workspace               = datamart_workspace_name, 
    dataset                 = datamart_dataset_name,
    new_dataset_workspace   = semantic_model_workspace_name,
    new_dataset             = semantic_model_name
)

## Rebind all reports using the old semantic model to the new Direct Lake semantic model

**IMPORTANT:** The data is not imported from the original datamart to the data warehouse. It is recommended to evaluate available data ingestion options, such as Dataflow Gen2, data pipelines, Copy Jobs, and more, to determine the optimal choice for your solution before rebinding existing reports.

In [ ]:
rep.report_rebind_all(
    dataset_workspace     = datamart_workspace_name,
    dataset               = datamart_dataset_name,
    new_dataset_workpace  = semantic_model_workspace_name,
    new_dataset           = semantic_model_name,
    report_workspace      = None
)

## Rebind reports one-by-one (optional)

**IMPORTANT:** The data is not imported from the original datamart to the data warehouse. It is recommended to evaluate available data ingestion options, such as Dataflow Gen2, Data pipelines, Copy jobs, and more, to determine the optimal choice for your solution before rebinding existing reports.

In [ ]:
report_workspace_name   = 'Datamart Migration' # Enter workspace where the report which you want to rebind is stored
report_name             = 'Sales Analysis'     # Enter report name which you want to rebind to the new Direct Lake model

rep.report_rebind(
    report_workspace    = report_workspace_name,
    report              = report_name,
    dataset_workspace   = semantic_model_workspace_name,
    dataset             = semantic_model_name
)

## Show unsupported objects

In [ ]:
dfT, dfC, dfR = directlake.show_unsupported_direct_lake_objects(dataset = datamart_dataset_name, workspace = datamart_workspace_name)

print('Calculated Tables are not supported...')
display(dfT)
print("Learn more about Direct Lake limitations here: https://learn.microsoft.com/power-bi/enterprise/directlake-overview#known-issues-and-limitations")
print('Calculated columns are not supported. Columns of binary data type are not supported.')
display(dfC)
print('Columns used for relationship must be of the same data type.')
display(dfR)